<a href="https://colab.research.google.com/github/kartheekknrs/number_plate_detection/blob/main/number_plate_detection_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BPXo2cyKUKZN95IBsvf2")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-11 in yolov11:: 100%|██████████| 20262/20262 [00:03<00:00, 6308.79it/s]


In [3]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [8]:
import os

# Path to your image folder
folder_path = "/content/License-Plate-Recognition-11/train/images"

# Common image extensions
image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp")

# Count files with image extensions
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(image_extensions)]

print(f"Total images: {len(image_files)}")


Total images: 7057


In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
from ultralytics import YOLO

model=YOLO("/content/yolo11m.pt")

model.train(data="/content/License-Plate-Recognition-11/data.yaml", imgsz=640, batch=8, epochs=20, workers=2, device=0)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo11m.pt, data=/content/License-Plate-Recognition-11/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, 

100%|██████████| 755k/755k [00:00<00:00, 149MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 371MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 14.1±4.2 MB/s, size: 24.9 KB)


...

                   all       2048       2195      0.969      0.937      0.963      0.677



20 epochs completed in 1.655 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:24<00:00,  5.33it/s]


                   all       2048       2195      0.971      0.929      0.965      0.677
Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d986d926f10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [14]:
import shutil
shutil.copytree('/content/runs', '/content/drive/MyDrive/runs_for_10000')


'/content/drive/MyDrive/runs_for_10000'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/numebr_plate_detection/runs/detect/train/weights/best.pt")

model.predict(source= "/content/PCS_Kennzeichenerk_handsfree.jpeg", show=True, save= True, conf=0.6, save_crop=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


image 1/1 /content/PCS_Kennzeichenerk_handsfree.jpeg: 448x640 1 LicensePlate, 49.7ms
Speed: 2.7ms preprocess, 49.7ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'LicensePlate'}
 obb: None
 orig_img: array([[[169, 163, 158],
         [170, 164, 159],
         [170, 164, 159],
         ...,
         [ 95,  97,  97],
         [196, 191, 190],
         [216, 208, 208]],
 
        [[169, 163, 158],
         [169, 163, 158],
         [169, 163, 158],
         ...,
         [150, 148, 148],
         [198, 193, 192],
         [211, 203, 203]],
 
        [[169, 163, 158],
         [169, 163, 158],
         [168, 162, 157],
         ...,
         [214, 206, 206],
         [205, 200, 199],
         [205, 200, 199]],
 
        ...,
 
        [[ 89,  93,  98],
         [ 90,  94,  99],
         [ 98, 102, 107],
         ...,
         [149, 159, 166],
         [149, 158, 167],
         [113, 122, 131]],
 
        [[ 87,  91,  96],
         [ 84,  88,  93],
         [ 90,  94,  99],
         ...,
         [ 8

In [ ]:
from itertools import count
from ultralytics import YOLO
import os
import cv2
from google.colab.patches import cv2_imshow
# Path to the input image
path1 = "/content/Datacluster_number_plates (1).jpg"

# Load the model
model = YOLO("/content/drive/MyDrive/numebr_plate_detection/runs/detect/train/weights/best.pt")

# Perform prediction
results = model.predict(source=path1, show=True, save=True, save_crop=True )
# Load the original image
image = cv2.imread(path1)

count=0
# Loop through the predictions to get the bounding boxes and crop the images
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0][:4].tolist())

        # Crop the image
        cropped_image = image[y1:y2, x1:x2]

        # Display the cropped image (optional)
        cv2_imshow( cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()